In [204]:
import json
import kfp
import kfp.dsl as dsl
import kfp.components as comp
from kfp.components import OutputPath

#### Pipeline Configurations

In [205]:
# replace this value with the value of the KFP host name
KFP_HOST_NAME = 'https://527bd6f955d1f721-dot-us-central2.pipelines.googleusercontent.com/'
# change caching policy of KFP run (useful in debug/dev mode)
disable_cache = False
#number of days hold out for validation
mltype = rf

In [206]:
client = kfp.Client(host=KFP_HOST_NAME)

#### Component - Download raw data

In [207]:
def download_raw_data(raw_data_path: str) -> str:
    '''load data from local storage'''
    import pandas as pd
    import numpy as np
    from google.cloud import storage
    import re

    
    """Downloads a blob from the bucket."""
    bucket_name = "bucket_de"
    source_blob_name = "raw/raw_data2.csv"
    destination_file_name = "tmp/raw_data2.csv"

    storage_client = storage.Client()
    
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)
    print('Downloaded data...')

    #import wget
    #wget -O bestand --post-data="variabele=waarde&variabele=waarde&...." http://projects.knmi.nl/klimatologie/daggegevens/getdata_dag.cgi
    #df = pd.read_csv('gs://bucket_de/raw/raw_data2,csv', error_bad_lines = False)
    df =  pd.read_csv("tmp/raw_data2.csv", error_bad_lines=False)

    
    def dfreplace(df, *args, **kwargs):
        s = pd.Series(df.values.flatten())
        s = s.str.replace(*args, **kwargs)
        return pd.DataFrame(s.values.reshape(df.shape), df.index, df.columns)
    
    weather_df = dfreplace(df, ',', '')
    
    for i in weather_df.columns:
        weather_df[i] = weather_df[i].astype(str)
        weather_df[i][weather_df[i].apply(lambda i: True if re.search('^\s*$', str(i)) else False)]=np.NaN

    print(weather_df.head())
    print('trying to write to GS')
    weather_df.to_parquet(raw_data_path, compression='GZIP')
    print('Done!')
    return raw_data_path

In [208]:
# create a KFP component
download_raw_data_op = comp.create_component_from_func(
    download_raw_data, output_component_file='download_raw_data.yaml', packages_to_install=['fastparquet', 'fsspec', 'gcsfs', "google-cloud-storage"])

#### Component - Feature processing

In [209]:
def feature_processing(raw_data_path: str, feature_data_path: str) -> str:
    '''calculate features for our machine learning model'''
    import pandas as pd
    from datetime import datetime

    # read dataframe
    weather_df = pd.read_parquet(raw_data_path)
    
    # create empty df to store feature
    weather_features_df = weather_df
    
    #create variables for years, months and days
    weather_features_df['YYYY'] = weather_features_df['YYYYMMDD,'].str.slice(0,4) #create a variable for years
    weather_features_df['MM'] = weather_features_df['YYYYMMDD,'].str.slice(4,6)#create a variable for months
    weather_features_df['DD'] = weather_features_df['YYYYMMDD,'].str.slice(6,8)
    for i in weather_features_df.columns:
            weather_features_df[i] = weather_features_df[i].astype(float, errors= 'ignore') 
    weather_features_df = weather_features_df.drop('YYYYMMDD,', axis=1)
    
    weather_features_df['TG_future'] = weather_features_df['TG,'].shift(periods = -1)
    
    #remove irrelevant features manually
    weather_features_df = weather_features_df.drop(columns = ['STN,','EV24', 'NG,', 'TN,', 'TNH,', 'TX,', 'TXH,', 'T10N,', 'T10NH,'])
    weather_features_df = weather_features_df.dropna()

    weather_features_df.to_parquet(feature_data_path, compression='GZIP')
    features_numbers = len(weather_features_df.columns) - 1
    print('Writing %s features' % (features_numbers))
    print('Done!')
    
    return feature_data_path

In [210]:
# create a KFP component
feature_processing_op = comp.create_component_from_func(
    feature_processing, output_component_file='feature_processing.yaml', packages_to_install=['fastparquet', 'fsspec', 'gcsfs', 'google-cloud-storage'])

#### Component - Predict model

In [211]:
def predict_weather_model(feature_data_path: str, regression_weather_model_path: str, validation_days: int) -> str:
    import json
    import pandas as pd
    import _pickle as cPickle # save ML model
    from google.cloud import storage # save the model to GCS
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import mean_absolute_error, mean_squared_error
    from sklearn.model_selection import train_test_split
    from urllib.parse import urlparse
    if model_type = rf:
        model_path = 'gs://bucket_de/model_store/rf/rf_rf.pickle'
    
    
    def predictions(model_path):
        '''this function evaluates a model on the validation dataset given just the model path'''
        parse = urlparse(url=model_path, allow_fragments=False)

        if parse.path[0] =='/':
            model_path = parse.path[1:]

        client = storage.Client()
        bucket = client.get_bucket(parse.netloc)
        blob = bucket.get_blob(model_path)
        if blob is None:
            raise AttributeError('No files to download') 
        model_bytestream = BytesIO(blob.download_as_string())
        model = cPickle.load(model_bytestream)
        predictions = model.predict(x)
        return predictions
    

    return regression_weather_model_path

In [212]:
# create a KFP component
train_regression_weather_model_op = comp.create_component_from_func(
    train_regression_weather_model, output_component_file='train_regression_weather_model.yaml', packages_to_install=['scikit-learn', 'fastparquet', 'fsspec', 'gcsfs', 'google-cloud-storage'])

#### Create and run KubeFlow pipeline

In [217]:
@dsl.pipeline(
  name='Weather regression',
  description='Predicting temperature in Eindoven using linear regression and random forest'
)
def weather_pipeline(raw_data_path, feature_data_path, regression_weather_model_path, rf_weather_model_path, disable_cache, validation_days):

  download_raw_data_task = download_raw_data_op(raw_data_path)
  feature_processing_task = feature_processing_op(download_raw_data_task.output, feature_data_path)
  train_regression_weather_model_task = train_regression_weather_model_op(feature_processing_task.output, regression_weather_model_path, validation_days)
  train_rf_weather_model_task = train_rf_weather_model_op(feature_processing_task.output, rf_weather_model_path, validation_days)
  eval_models_task = eval_models_op(feature_processing_task.output, train_regression_weather_model_task.output, train_rf_weather_model_task.output, validation_days)

  if disable_cache:
      download_raw_data_task.execution_options.caching_strategy.max_cache_staleness = "P0D"
      feature_processing_task.execution_options.caching_strategy.max_cache_staleness = "P0D"
      train_regression_weather_model_task.execution_options.caching_strategy.max_cache_staleness = "P0D"
      train_rf_weather_model_task.execution_options.caching_strategy.max_cache_staleness = "P0D"
      eval_models_task.execution_options.caching_strategy.max_cache_staleness = "P0D"
    
# Specify argument values for pipeline run.
arguments = {'raw_data_path': 'gs://bucket_de/raw/weater.parquet',
            'feature_data_path': 'gs://bucket_de/raw/feature_data/weater_features.parquet',
            'regression_weather_model_path': 'gs://bucket_de/model_store/vanilla/vanilla_rf.pickle',
            'rf_weather_model_path': 'gs://bucket_de/model_store/rf/rf_rf.pickle',
            'disable_cache': disable_cache,
            'validation_days': validation_days,
            }


    
# Create a pipeline run, using the client you initialized in a prior step.
#client.create_run_from_pipeline_func(weather_pipeline, arguments=arguments)
if __name__ == '__main__':
    kfp.compiler.Compiler().compile(p, __file__ + '.yaml')

RunPipelineResult(run_id=2eed059f-200c-4bb8-95c3-fd61c8fece8e)